# Basics of Pydantic

Pydantic models are the foundation of data validation in Python. They use python type annotations to define the structure and validate data at runtime.

#### Creating and Using Pydantic model

Let us explore the basic model creation with several examples

In [1]:
from pydantic import BaseModel

In [2]:
class Person(BaseModel):
    name:str
    age:int
    city:str

person = Person(name = "Charan",age = 23,city = "Hyderabad")
print(person)
print(type(person))

name='Charan' age=23 city='Hyderabad'
<class '__main__.Person'>


### Differentiating dataclass and pydantic

In dataclass, there is no data validation i.e the property which is defined as string in the class can hold a value of integer when an object is created of that class.

But when we use pydantic and the class inherits the BaseModel, data validation occurs, so the property defined as string in the class can only hold string. Giving any other data type value will lead to an error.

In [3]:
# Using dataclass
from dataclasses import dataclass

@dataclass
class Person():
    name:str
    age:int
    country:str

charan = Person(name="Charan",age="Twenty three",country="India")
print(charan)

Person(name='Charan', age='Twenty three', country='India')


In [4]:
# Using pydantic BaseMode

from pydantic import ValidationError

class Person(BaseModel):
    name:str
    age:int
    country:str

try:
    charan = Person(name="Charan",age="Twenty three",country="India")
    print(charan)
except ValidationError  as e:
    print(e)

1 validation error for Person
age
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='Twenty three', input_type=str]
    For further information visit https://errors.pydantic.dev/2.11/v/int_parsing


### Model with Optional Fields

- Optional[Type] : indicates field that can be None
- Default value (=None or =True) : Makes the field optional
- Required fields must still be provided
- Pydantic even validates data types of optional fields if values are provided.

In [5]:
from typing import Optional

class Employee(BaseModel):
    employee_id:int
    name:str
    department:str
    salary:Optional[float] = None  #Optional with default value as None
    is_active:Optional[bool]=True  #Optional with default value as None

In [6]:
# Giving only field that are not optional
charan = Employee(name="Charan",employee_id=244,department="AI")
print(charan)

employee_id=244 name='Charan' department='AI' salary=None is_active=True


In [7]:
# Giving all fields
charan = Employee(name="Charan",employee_id=244,department="AI",salary=80000.0,is_active=False)
print(charan)

employee_id=244 name='Charan' department='AI' salary=80000.0 is_active=False


In [8]:
# Typecasting ability of pydantic
charan = Employee(name="Charan",employee_id=244,department="AI",salary=80000,is_active=False)
print(charan)

employee_id=244 name='Charan' department='AI' salary=80000.0 is_active=False


### Lists as data type in classes

In [9]:
from typing import List 

class Classroom(BaseModel):
    room_number:str
    students:List[str]
    capacity:int

In [10]:
classroom = Classroom(
    room_number="A407",
    students=["Sachin","Sehwag","Dravid"],
    capacity=100
)
print(classroom)

room_number='A407' students=['Sachin', 'Sehwag', 'Dravid'] capacity=100


In [11]:
# Typecasting ability of Pydantic
# Let us give tuple instead of list to understand what happens

classroom = Classroom(
    room_number="A408",
    students=("Ponting","Hayden","Gilchrist"),
    capacity=50
)
print(classroom)

room_number='A408' students=['Ponting', 'Hayden', 'Gilchrist'] capacity=50


### Complex structures

In [12]:
class Address(BaseModel):
    street:str
    city:str
    zip_code:int

class Customer(BaseModel):
    customer_id:int
    name:str
    address:Address

In [13]:
try:
    customer = Customer(
        customer_id=2,
        name="Rohit",
        address={
            "street":"Boravali",
            "city":"Mumbai",
            "zip_code":500098
        }
    )
    print(customer)
except ValueError as e:
    print(e)

customer_id=2 name='Rohit' address=Address(street='Boravali', city='Mumbai', zip_code=500098)


In [14]:
# Passing string in the place of integer
# String will be type-casted into integer by Pydantic
# But integer will not be type-casted into string by pydantic

customer = Customer(
    customer_id=1,
    name="Charan",
    address={
        "street":"Neeladri",
        "city":"Bangalore",
        "zip_code":"560100"
    }
)
print(customer)

customer_id=1 name='Charan' address=Address(street='Neeladri', city='Bangalore', zip_code=560100)


### Pydantic fields : Customization and Constraints

The field function in Pydantic enhances model fields beyond basic type hints by allowing you to specify validation rules, default values , aliases and more. 

In [15]:
from pydantic import Field

class Item(BaseModel):
    name:str=Field(min_length=2,max_length=50)
    price:float=Field(gt=0,le=1000)
    quantity:int=Field(ge=0)

In [16]:
item = Item(name="Book",price=500,quantity=2)
print(item)

name='Book' price=500.0 quantity=2


In [17]:
# If something doesn't satisfy constraints

try:
    item = Item(name="Book",price=-500,quantity=3)
    print(item)
except ValueError as e:
    print(e)

1 validation error for Item
price
  Input should be greater than 0 [type=greater_than, input_value=-500, input_type=int]
    For further information visit https://errors.pydantic.dev/2.11/v/greater_than


### Pydantic fields : Optional and defaults values

We can also specify if something is optional inside the Pydantic field, give description to the variable and also give default values

In [18]:
class User(BaseModel):
    username:str=Field(...,description="Unique username of the user")
    age:int=Field(default=18,description="User age, defaults to 18")
    email:str=Field(default_factory=lambda:"user@example.com",description="Default email address is user@example.com")

In [19]:
user1 = User(username="Charan",age=23)
print(user1)

username='Charan' age=23 email='user@example.com'


In [20]:
user2 = User(username="Rohit",age=38,email="rohit264@gmail.com")
print(user2)

username='Rohit' age=38 email='rohit264@gmail.com'


In [21]:
# Importance of description

print(User.model_json_schema())

{'properties': {'username': {'description': 'Unique username of the user', 'title': 'Username', 'type': 'string'}, 'age': {'default': 18, 'description': 'User age, defaults to 18', 'title': 'Age', 'type': 'integer'}, 'email': {'description': 'Default email address is user@example.com', 'title': 'Email', 'type': 'string'}}, 'required': ['username'], 'title': 'User', 'type': 'object'}
